In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data = pd.read_csv('nyse/fundamentals.csv', parse_dates=['Period Ending'],
                                              infer_datetime_format=True)

# data looks clean
print('Non-numeric columns: ', list(data.select_dtypes(exclude='number').columns))
data = data.drop(columns=['Unnamed: 0'])
data.head()

Non-numeric columns:  ['Ticker Symbol', 'Period Ending']


,Ticker Symbol,Period Ending,Accounts Payable,Accounts Receivable,Add'l income/expense items,After Tax ROE,Capital Expenditures,Capital Surplus,Cash Ratio,Cash and Cash Equivalents,...,Total Current Assets,Total Current Liabilities,Total Equity,Total Liabilities,Total Liabilities & Equity,Total Revenue,Treasury Stock,For Year,Earnings Per Share,Estimated Shares Outstanding
0,AAL,2012-12-31,3.068000e+09,-222000000.0,-1.961000e+09,23.0,-1.888000e+09,4.695000e+09,53.0,1.330000e+09,...,7.072000e+09,9.011000e+09,-7.987000e+09,2.489100e+10,1.690400e+10,2.485500e+10,-367000000.0,2012.0,-5.60,3.350000e+08
1,AAL,2013-12-31,4.975000e+09,-93000000.0,-2.723000e+09,67.0,-3.114000e+09,1.059200e+10,75.0,2.175000e+09,...,1.432300e+10,1.380600e+10,-2.731000e+09,4.500900e+10,4.227800e+10,2.674300e+10,0.0,2013.0,-11.25,1.630222e+08
2,AAL,2014-12-31,4.668000e+09,-160000000.0,-1.500000e+08,143.0,-5.311000e+09,1.513500e+10,60.0,1.768000e+09,...,1.175000e+10,1.340400e+10,2.021000e+09,4.120400e+10,4.322500e+10,4.265000e+10,0.0,2014.0,4.02,7.169154e+08
3,AAL,2015-12-31,5.102000e+09,352000000.0,-7.080000e+08,135.0,-6.151000e+09,1.159100e+10,51.0,1.085000e+09,...,9.985000e+09,1.360500e+10,5.635000e+09,4.278000e+10,4.841500e+10,4.099000e+10,0.0,2015.0,11.39,6.681299e+08
4,AAP,2012-12-29,2.409453e+09,-89482000.0,6.000000e+05,32.0,-2.711820e+08,5.202150e+08,23.0,5.981110e+08,...,3.184200e+09,2.559638e+09,1.210694e+09,3.403120e+09,4.613814e+09,6.205003e+09,-27095000.0,2012.0,5.29,7.328355e+07


In [4]:
# shifting year for companies that report last year earnings in january
data['year'] = data['Period Ending'].map(lambda d: d.year if d.month > 1 else d.year - 1)

# splitting by year
data_2012 = data[data.year == 2012]
data_2013 = data[data.year == 2013]
data_2014 = data[data.year == 2014]
data_2015 = data[data.year == 2015]
data_2016 = data[data.year == 2016]

# using years 2013-2015 as features
data_wide = pd.merge(data_2013, data_2014, on='Ticker Symbol', how='outer', suffixes=(' 2013', ' 2014'))
data_wide = pd.merge(data_wide, data_2015, on='Ticker Symbol', how='outer', suffixes=('', ' 2015'))

# 2016 revenue is our target variable
data_revenue = pd.DataFrame(data_2016[['Ticker Symbol', 'Total Revenue']])
data_wide = pd.merge(data_wide, data_revenue , on='Ticker Symbol', how='outer', suffixes=('', ' 2016'))

# only leaving non-null records, ~30k of them
data_wide = data_wide.dropna()
data_wide.size

33512

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# normalizing
scaler = StandardScaler()
numerics = data_wide.select_dtypes(include='number')
X_raw_df = numerics.drop(['Total Revenue 2016'], axis=1)
X_raw = X_raw_df.values
X = scaler.fit_transform(X_raw)
y = numerics['Total Revenue 2016'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11)

# feature selection
model = linear_model.LinearRegression()
rfe = RFE(model, 3)
X_rfe = rfe.fit_transform(X_train, y_train)

cols_to_drop = []
column_names = []
column_indices = []
all_columns = X_raw_df.columns

for i, mask in enumerate(rfe.support_):
  if mask:
    column_names.append(all_columns[i])
    column_indices.append(i)
  else:
    cols_to_drop.append(i)

print('Selected columns:', column_names)
# ['Depreciation 2013', 'Depreciation', 'Total Revenue']
    
X_test_df = pd.DataFrame(X_test)
X_test_df = X_test_df.drop(columns=cols_to_drop)
X_test_slim = X_test_df.values

model.fit(X_rfe, y_train)
y_pred = model.predict(X_test_slim)
print('Mean squared error:', mean_squared_error(y_test, y_pred))

# AAPL depreciation 2016:   10505
# AAPL depreciation 2017:   10157
# AAPL total revenue 2017:  229234
aapl_raw = pd.DataFrame(X_raw_df.iloc[[0]].copy())
aapl_raw['Depreciation 2013'] = 10505000000
aapl_raw['Depreciation'] = 10157000000
aapl_raw['Total Revenue'] = 229234000000
aapl_all = scaler.transform(aapl_raw.values).ravel()
aapl_selected = np.array([aapl_all[i] for i in column_indices]).reshape(1, 3)
aapl_pred = model.predict(aapl_selected)

print(aapl_pred)


Selected columns: ['Depreciation 2013', 'Depreciation', 'Total Revenue']
Mean squared error: 8.18540336933153e+19
[2.32974837e+11]
